# Basic example

Multiplication of two integer numbers is pretty simple.
Additional information can be got from Python [documentation](https://docs.python.org/3/library/ctypes.html).

## Write C code

We want to multiply two integers and get integer as the result
```c
int mul(int a, int b) {
    return a * b;
}
```

Notice, that it's not mandatory to have `main` entry point function

## Compile shared library (dll in Windows)

```bash
gcc -shared basics.c -o lib_basics.so
```

## Load created library in Python code

In [1]:
from ctypes import cdll

basics = cdll.LoadLibrary('./lib_basics.so')

## Usage

Now it's pretty simple to call `mul` function just like it was a property of module `basics`

In [2]:
basics.mul(2, 5)

10

See? It's easy

# Dot product of two arrays

Array is a block of memory splitted into chunks of a single type and it's easy to use C arrays with `ctypes`

## C function

Just iterate through each element of arrays, multiply corresponding ones and summarize the result
```c
#include <stdlib.h>

int dot(int *a, int *b, size_t length) {
    int result = 0;
    while (length --> 0) {
        result += a[length] * b[length];
    }
    return result;
}
```

## Create arrays

We should import `int` data type from `ctypes`

In [3]:
from ctypes import c_int

Say, it's needed to multiply 3-dimensional vectors

In [4]:
first = (c_int * 3)(1, 2, 3)

We can create an alias for this data type and use it

In [5]:
vector3D = c_int * 3
second = vector3D(4, 5, 6)

## Call the function

In [6]:
c_result = basics.dot(first, second, 3)
python_result = sum(a * b for a, b in zip([1, 2, 3], [4, 5, 6]))
print('C returned', c_result, 'and Python returned', python_result)

C returned 32 and Python returned 32


In [7]:
basics.dot((c_int*1)(2), (c_int*1)(*[3]), 1)

6

Following examples will cause errors

In [8]:
try:
    vector3D([1, 2, 3])
except:
    print('You cannot pass lists')
try:
    vector3D(0, 1, 2, 3)
except:
    print('Forbidden to provide more elements than it should accept')   

You cannot pass lists
Forbidden to provide more elements than it should accept


# Available types

Following types can be used to pass arguments to `C` functions

| ctypes       | type                                    | C type  Python type |
| -----------  | --------------------------------------  | ------------------- |
| c_bool       | _Bool                                   | bool (1) |
| c_char       | char                                    | 1-character bytes object |
| c_wchar      | wchar_t                                 | 1-character string |
| c_byte       | char                                    | int |
| c_ubyte      | unsigned                                | char   int |
| c_short      | short                                   | int |
| c_ushort     | unsigned                                | short  int |
| c_int        | int                                     | int |
| c_uint       | unsigned                                | int    int |
| c_long       | long                                    | int |

| ctypes       | type                                    | C type  Python type |
| -----------  | --------------------------------------  | ------------------- |
| c_ulong      | unsigned long                           | int |
| c_longlong   | __int64 or long long                    | int |
| c_ulonglong  | unsigned __int64 or unsigned long long  | int |
| c_size_t     | size_t                                  | int |
| c_ssize_t    | ssize_t or Py_ssize_t                   | int |
| c_float      | float                                   | float |
| c_double     | double                                  | float |
| c_longdouble | long double                             | float |
| c_char_p     | char * (NUL terminated)                 | bytes object or None |
| c_wchar_p    | wchar_t * (NUL terminated)              | string or None |
| c_void_p     | void *                                  | int or None |

# Change the long passed to your function

Long number in Python represented as an array of longs:
```
{d0, d1, d2, ...}
```

We can change each one of them, so why not?

## Prepare shared library

C function:
```c
#include <Python.h>

int set_long(PyLongObject* o, long new_value,

             size_t digit) {
             
    o->ob_digit[digit] = new_value;
    
    return 0;
}
```

File should be compiled to shared library (dll in Windows).

Makefile for Linux:
```
FLAGS=-shared
LIBRARIES=-I/usr/include/python3.4
BUILD_LIBRARY=gcc $(FLAGS) $(LIBRARIES)
all:
    $(BUILD_LIBRARY) setters.c -o lib_setters.so
```

## Use shared library

It's handy to create Python wrapper for this C function

In [9]:
from ctypes import cdll, c_long, c_size_t, c_voidp

setters = cdll.LoadLibrary('./lib_setters.so')

def change_long(a, b=0, digit=0):
    setters.set_long(c_voidp(id(a)), c_long(b), c_size_t(digit))

Don't forget, that Python interpreter will not create new objects for little integers like `0`, so we should avoid assigning of new values to such numbers, because they will be changed in all places where they're used

In [10]:
from ctypes import c_long, c_size_t, c_voidp

def change_long(a, b=0, digit=0):
    args = (a, b, digit)
    if not all(type(a) is int for a in args):
        raise TypeError('All parameters should be of type "int", '
                        'but {} provided'.format(map(type, args)))
    if a + 0 is a:
        raise ValueError('No way. You don\'t want to break '
                         'your interpreter, right?')
    setters.set_long(c_voidp(id(a)), c_long(b), c_size_t(digit))

Let's recall, that we cannot change the value of integers inside of Python functions

In [11]:
def variable_info(text, variable):
    print('{:^30}: {:#05x} ({:#x})'.format(text, variable, id(variable)))

def foo(a, new_value):
    a = new_value

a = 2**10
variable_info('Before function call', a)
foo(a, 5)
variable_info('After function call', a)

     Before function call     : 0x400 (0x7fc66da16290)
     After function call      : 0x400 (0x7fc66da16290)


Now forget it and take a look on what we've done

In [12]:
a = 2**10
b = a
variable_info('Before function call', a)
change_long(a, 2, 0)
variable_info('After function call', a)
variable_info('What\'s about b? Here it is', b)

     Before function call     : 0x400 (0x7fc66da16d90)
     After function call      : 0x002 (0x7fc66da16d90)
  What's about b? Here it is  : 0x002 (0x7fc66da16d90)


![hahaha](./lied.jpg)